In [1]:
# from google.colab import drive
# drive.mount('/drive')

In [2]:
# !pip install transformers
# !pip install sentencepiece

In [3]:
# cr loanwords = ['chanson','auteur','a la mode','académie','après','bourgeois','burlesque','depanneur','detente','ennui','faux pas','fete','fin de siecle','gite','jongleur','lycee','peloton','proletariat','rendezvous','salon']
# fr-en loanwords = ['beige','brochure','cafe','camaraderie','cliche','clique','concierge','coup','detour','elite','ensemble','etiquette','facade','fine','foyer','naive','nuance','physique','rapport','reservoir']
# literal translations = ['academy','after','author','boredom','cottage','end of century','fashionable','high school','misstep','mockery','pack','party','relaxation','repairman','room','song','minstrel','working class','meeting','middle class']
# non-borrowed words = ['fire','nose','water','tongue','blood','bone','rain','name','sweet','fly','night','ear','house','bitter','salt','knee','new','old','thick','long']
# en-fr loanwords = ['marketing','business','vintage','spoiler','casting','bestseller','pitch','cool','loser','coach','email','brunch','digital','leader','hacker','manager','jogging','yes','match','shopping']
# non-borrowed-words-fr = ['feu','nez','eau','langue','sang','os','pluie','nom','doux','mouche','nuit','oreille','maison','amer','sel','genou','nouveau','ancien','epais','long']

In [4]:
alternative_spellings = {'a la mode':'à la mode','académie':'academie','après':'apres','depanneur':'dépanneur','détente':'detente','faux pas':'faux-pas','fete':'fête','fin de siecle':'findesiecle','gite':'gîte','lycee':'lycée','rendezvous':'rendez vous','cafe':'café','cliche':'cliché','elite':'élite','etiquette':'étiquette','facade':'façade','naive':'naïve','end of century':'end-of-century','middle-class':'middle class','bestseller':'best seller','epais':'épais'}

In [2]:
model_name = 'bert-base-multilingual-uncased'
words = ['chanson','song']
# k_values = [10,25,50,100,150,200]
max_k = 50
language = 'fr'

In [3]:
from transformers import BertTokenizer, BertForMaskedLM,CamembertForMaskedLM, CamembertTokenizer
import sentencepiece

if model_name == "camembert/camembert-base-ccnet":
  model = CamembertForMaskedLM.from_pretrained("camembert/camembert-base-ccnet",output_hidden_states = True)
  tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-ccnet")

else:
  tokenizer = BertTokenizer.from_pretrained(model_name)
  model = BertForMaskedLM.from_pretrained(model_name,output_hidden_states = True)


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
import torch

def pre_process_sentence(sentence,target_tokens): # return a tensor with target word masked

  # Tokenise input text
  tokenized_text = tokenizer.tokenize(sentence)

  # Create attention mask
  attention_mask = [1] * len(tokenized_text)

  target_indices = []
  for item in target_tokens:
    for i in range(len(tokenized_text)):
      if tokenized_text[i:i+len(item)] == item:
        tokens_indices = [j for j in range(i,i+len(item)) if i < (512-len(item))]
        for k in tokens_indices:
          if k not in target_indices:
            target_indices.append(k)

  for i in target_indices:
      tokenized_text[i] = '[MASK]'
      attention_mask[i] = 0

  input_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)], dtype=torch.long)
  attention_mask_tensor = torch.tensor([attention_mask], dtype=torch.long)
  input_tensor = input_tensor.to(device)
  attention_mask_tensor = attention_mask_tensor.to(device)

  return input_tensor, attention_mask_tensor, target_indices


In [ ]:
import torch
def get_predicted_words(input_tensor,attention_mask_tensor,target_indices,model,tokenizer,k):

  if input_tensor.size(1) > 512:
    # Truncate the input tensor to fit within the model's maximum size
    input_tensor = input_tensor[:, :512]
    attention_mask_tensor = attention_mask_tensor[:, :512]

    # model = model.to(torch.long)

    # Feed the input tensor to the model and predict words
  with torch.no_grad():
      output = model(input_tensor,attention_mask = attention_mask_tensor)

  top_k_predictions = {}
  for i in target_indices:
    masked_token_logits = output.logits[0, i]
    masked_token_probs = torch.softmax(masked_token_logits, dim=-1)
    top_k_tokens = torch.topk(masked_token_probs, k=k)
    predictions = [(tokenizer.decode(token.item())) for token in top_k_tokens.indices]
    # top_k_predictions.extend([tokenizer.decode(token.item()) for token in top_k_tokens.indices])
    top_k_predictions[i]=[element.replace(' ', '') for element in predictions]


  return top_k_predictions


In [ ]:
cr_loanwords = ['chanson','auteur','a la mode','académie','après','bourgeois','burlesque','depanneur','detente','détente','ennui','faux pas','fete','fin de siecle','gite','jongleur','lycee','lycée','peloton','proletariat','rendezvous','salon']
fr_en_loanwords = ['beige','brochure','cafe','camaraderie','cliche','clique','concierge','coup','detour','elite','ensemble','etiquette','facade','fine','foyer','naive','nuance','physique','rapport','reservoir']
literal_translations = ['academy','after','author','boredom','cottage','end of century','fashionable','high school','misstep','mockery','pack','party','relaxation','repairman','room','song','minstrel','working class','meeting','middle class']
non_borrowed_words = ['fire','nose','water','tongue','blood','bone','rain','name','sweet','fly','night','ear','house','bitter','salt','knee','new','old','thick','long']
en_fr_loanwords = ['marketing','business','vintage','spoiler','casting','bestseller','pitch','cool','loser','coach','email','brunch','digital','leader','hacker','manager','jogging','yes','match','shopping']
non_borrowed_words_fr = ['feu','nez','eau','langue','sang','os','pluie','nom','doux','mouche','nuit','oreille','maison','amer','sel','genou','nouveau','ancien','epais','long']

types = {}
for w in cr_loanwords:
  types[w]= 'cr loanword'

for w in fr_en_loanwords:
  types[w] = 'fr-en loanword'

for w in literal_translations:
  types[w] = 'literal translation'

for w in non_borrowed_words:
  types[w] = 'non borrowed word'

for w in en_fr_loanwords:
  types[w] = 'en-fr loanword'

for w in non_borrowed_words_fr:
  types[w] = 'non borrowed word fr'

In [ ]:
import matplotlib.pyplot as plt
import csv

for word in words:

  token_instances = 0

  if word in alternative_spellings.keys(): # checking if there are alternative spellings and adding these to be masked
    target_words = [word,word+'s',alternative_spellings[word],alternative_spellings[word]+'s']
  else:
    target_words = [word,word+'s']

  # Tokenise target word, plurals and alternative spellings
  target_tokens = []
  for target in target_words:
    if tokenizer.tokenize(target.lower()) not in target_tokens: # alternative spellings may be same when tokenised
      target_tokens += [tokenizer.tokenize(target.lower())] # producing a list of tuples of tokens, where each tuple is a word

  print('Target word: ',word)

  with open('Sentence datasets/'+word+'_'+language+'_unique.txt','r') as final_list:
    sentences = final_list.readlines()
    sentences = [str(sentence.strip()) for sentence in sentences if sentence.strip()]  # Skip empty lines
    sentences = sentences[:2000]

  word_frequency = {}
  runtime_errors = 0
  for sentence in sentences:

    input_tensor,attention_mask_tensor,target_indices = pre_process_sentence(sentence,target_tokens)

    token_instances += len(target_indices) # adding the number of tokens
    # print('Sentence number: ',sentences.index(sentence),'Number of masked tokens:',len(target_indices))
    if token_instances >= (500*len(target_tokens[0])):
      break

    if input_tensor!= None:
      if len(input_tensor)!= 0:
        try:
          predictions = get_predicted_words(input_tensor,attention_mask_tensor,target_indices,model,tokenizer,max_k)
        except RuntimeError:
          runtime_errors += 1
        for prediction_list in predictions.values():
          for p in prediction_list:
            if p in word_frequency.keys():
              word_frequency[p]+=1
            else:
              word_frequency[p]=1


  sorted_word_frequency = {k: v for k, v in sorted(word_frequency.items(), key=lambda item: item[1], reverse=True)}

  with open('Results/MLM_Results/masked_word_guesses_'+model_name+'_'+language+'.csv','a') as GUESSES:
    guesseswriter = csv.writer(GUESSES)
    guesseswriter.writerow([word, types[word],tokenizer.tokenize(word),sorted_word_frequency])



Target word:  oreille
Target word:  maison
Target word:  amer
Target word:  sel
Target word:  genou
Target word:  nouveau
Target word:  ancien
Target word:  epais
Target word:  long


In [ ]:
print(sorted_word_frequency)

{'long': 367, 'grand': 219, 'tout': 219, 'le': 211, 'petit': 208, 'meme': 208, 'moyen': 204, 'plus': 201, 'temps': 201, 'de': 200, 'bon': 194, 'terme': 190, 'du': 181, 'niveau': 178, 'premier': 176, 'titre': 175, 'monde': 172, 'haut': 167, 'son': 167, 'ce': 167, 'milieu': 162, 'fait': 159, 'cœur': 155, 'un': 153, 'simple': 152, 'nom': 152, 'a': 150, 'second': 140, 'debut': 140, 'seul': 139, 'cours': 137, 'reste': 135, 'dernier': 134, '-': 134, 'gros': 132, 'jour': 130, 'court': 125, 'fort': 123, 'bien': 123, 'en': 122, 'sens': 122, 'nouveau': 121, 'cote': 120, 'bout': 119, 'lieu': 118, 'bord': 118, 'tour': 118, 'fond': 118, 'moins': 116, 'developpement': 115, 'moment': 114, 'la': 112, 'point': 111, 'centre': 111, 'au': 110, 'service': 109, 'double': 107, 'stade': 106, 'sein': 106, 'sans': 105, 'pays': 105, 'large': 101, 'chemin': 99, 'meilleur': 97, 'fil': 97, 'dela': 97, 'site': 96, 'ton': 95, 'tel': 94, ',': 94, 'vieux': 92, 'mauvais': 90, 'depart': 90, 'sommet': 89, 'nombre': 88, 'd

In [ ]:
total_tokens = sum(sorted_word_frequency.values())